In [ ]:

import os
import json
from typing import List, Dict
from openai import AzureOpenAI
from pymilvus import MilvusClient
from retriever import MedCPTRetriever
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize clients
retrieval_client = MilvusClient(uri=os.getenv("MILVUS_URI"))
# Initialize the retriever
retriever = MedCPTRetriever()

# Sample medical question
question = "A 45-year-old woman presents with fatigue, weight gain, and cold intolerance. Laboratory tests reveal elevated TSH and low free T4. What is the most likely diagnosis?"
choices = {
    "A": "Hyperthyroidism",
    "B": "Hypothyroidism",
    "C": "Adrenal insufficiency",
    "D": "Diabetes mellitus"
}

# Format the question with choices
def format_question(question: str, options: Dict[str, str]) -> str:
    text = f"{question}\n\n"
    for choice, option in options.items():
        text += f"({choice}) {option}\n"
    return text

formatted_query = format_question(question, choices)
print("Formatted Query:")
print(formatted_query)

# Define allowed source

Formatted Query:
A 45-year-old woman presents with fatigue, weight gain, and cold intolerance. Laboratory tests reveal elevated TSH and low free T4. What is the most likely diagnosis?

(A) Hyperthyroidism
(B) Hypothyroidism
(C) Adrenal insufficiency
(D) Diabetes mellitus


Retrieved 100 documents

Top 5 Documents after Reranking:

Document 1:
InternalMed_Harrison. Endocrine Disorders Fatigue, sometimes in association with true muscle weakness, can be a heralding symptom of hypothyroidism, particularly in the context of hair loss, dry skin, cold intolerance, constipation, and weight gain. Fatigue in association with heat intolerance, swea...

Document 2:
First_Aid_Step2. Presents with weakness, fatigue, cold intolerance, constipation, weight gain, depression, menstrual irregularities, and hoarseness. Exam may reveal dry, cold, puffy skin accompanied by edema, bradycardia, and delayed relaxation of DTRs.

See Table 2.3-3.

Uncomplicated hypothyroidis...

Document 3:
InternalMed_Harrison.

In [2]:
allowed_sources = ['cpg', 'recop', 'textbooks', 'statpearls']

# Retrieve documents
try:
    # Retrieve documents with filtering by source
    retrieved_docs = retriever.retrieve_filtered_sources(
        formatted_query,
        retrieval_client,
        allowed_sources,
        topk=100
    )
    
    print(f"\nRetrieved {len(retrieved_docs)} documents")
    
    # Rerank the documents
    reranked_docs = retriever.rerank(
        formatted_query,
        retrieved_docs,
    )
    rerank_time = time.time() - start_rerank
    
    print(f"Reranked documents in {rerank_time:.2f} seconds")
    
    # Get top 5 documents after reranking
    top_docs = reranked_docs[:5]
    
    print("\nTop 5 Documents after Reranking:")
    for i, doc in enumerate(top_docs):
        print(f"\nDocument {i+1}:")
        print(f"{doc[:300]}...")  # Print first 300 characters of each document
    
except Exception as e:
    print(f"Error during retrieval: {e}")



Retrieved 400 documents in 1.93 seconds
Reranked documents in 25.25 seconds

Top 5 Documents after Reranking:

Document 1:
InternalMed_Harrison. Endocrine Disorders Fatigue, sometimes in association with true muscle weakness, can be a heralding symptom of hypothyroidism, particularly in the context of hair loss, dry skin, cold intolerance, constipation, and weight gain. Fatigue in association with heat intolerance, swea...

Document 2:
Physiology, Thyroid Function -- Pathophysiology. Hypothyroidism is an endocrine disorder with resultant under-production of thyroid hormone. Common symptoms of hypothyroidism include cold intolerance and weight gain due to decreased basal metabolic rate and thermogenesis, depression, fatigue, decrea...

Document 3:
Thyrotoxicosis -- Differential Diagnosis. Most patients will present with symptoms of fatigue, palpitations, weight loss, diaphoresis, heat intolerance. These patients are generally stable for outpatient workup for the underlying etiology.  A

In [2]:
retrieval_client.list_collections()

['cpg', 'recop', 'textbooks', 'statpearls']

In [ ]:
# Delete all collections in Milvus
try:
    collections = retrieval_client.list_collections()
    print("Deleting the following collections:")
    for collection in collections:
        print(f"- {collection}")
        try:
            retrieval_client.drop_collection(collection)
            print(f"Successfully deleted collection: {collection}")
        except Exception as e:
            print(f"Error deleting collection '{collection}': {e}")
    
    # Verify collections were deleted
    remaining_collections = retrieval_client.list_collections()
    if remaining_collections:
        print(f"\nRemaining collections: {remaining_collections}")
    else:
        print("\nAll collections have been successfully deleted.")
except Exception as e:
    print(f"Error during collection deletion process: {e}")


In [4]:
# Check what Milvus collections are available
try:
    # List all collections in Milvus
    collections = retrieval_client.list_collections()
    print("\nAvailable Milvus Collections:")
    for collection in collections:
        print(f"- {collection}")
    
    # Get more details about each collection
    print("\nCollection Details:")
    for collection in collections:
        try:
            # Load the collection to get information
            retrieval_client.load_collection(collection)
            stats = retrieval_client.get_collection_stats(collection)
            entity_count = retrieval_client.count_entities(collection)
            print(f"\nCollection: {collection}")
            print(f"Entity Count: {entity_count}")
            print(f"Stats: {stats}")
        except Exception as e:
            print(f"Error getting details for collection '{collection}': {e}")
        finally:
            # Release the collection
            retrieval_client.release_collection(collection)
            
except Exception as e:
    print(f"Error listing collections: {e}")



Available Milvus Collections:
- textbooks_2
- statpearls_2
- cpg_2
- recop_2

Collection Details:
Error getting details for collection 'textbooks_2': 'MilvusClient' object has no attribute 'count_entities'
Error getting details for collection 'statpearls_2': 'MilvusClient' object has no attribute 'count_entities'
Error getting details for collection 'cpg_2': 'MilvusClient' object has no attribute 'count_entities'
Error getting details for collection 'recop_2': 'MilvusClient' object has no attribute 'count_entities'
